In [1]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv
import os

# Step 1: Load API key from .env file
load_dotenv()
api_key_01 = os.getenv("HUGGINGFACE_API_KEY")

# Step 2: Select a parser for the output (in this case, a simple string parser)
str_parser = StrOutputParser()


llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation",
    huggingfacehub_api_token=api_key_01,
    temperature=0.1 # Lower temperature for better JSON generation
)

model = ChatHuggingFace(llm=llm)


class FeedbackAnalysis(BaseModel):
    sentiment: Literal['positive', 'negative', 'neutral'] = Field(..., description="The sentiment of the feedback")
    
pyd_parser = PydanticOutputParser(pydantic_object=FeedbackAnalysis)

prompt_01 = PromptTemplate(
    template="""Classify the sentiment of the following feedback text as positive, negative, or neutral.

{format_instructions}

IMPORTANT: Return ONLY the raw JSON output. Do not include any python code, markdown formatting (like ```json), or explanations.

Feedback: {feedback}""",
    input_variables=["feedback"],
    partial_variables={"format_instructions": pyd_parser.get_format_instructions()}
)

classifier_chain = prompt_01 | model | pyd_parser

prompt_02 = PromptTemplate(
    template='Write an exact appropiate response of the following positive feedback, dont give me feedback options\n {feedback}',
    input_variables=["feedback"]
)

prompt_03 = PromptTemplate(
    template='Write an exact appropiate response of the following negitive feedback, dont give me feedback options\n {feedback}',
    input_variables=["feedback"]
)

branch_chain = RunnableBranch(
    (lambda x: x.sentiment == "positive", prompt_02 | model | str_parser),
    (lambda x: x.sentiment == "negative", prompt_03 | model | str_parser),
    RunnableLambda(lambda x: "Neutral feedback, no response needed")
)


chain = classifier_chain | branch_chain

result = chain.invoke({"feedback": "The product is just a holly shit, it broke after one use and the customer service was terrible!"})

print("Final Response: ", result)

C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Final Response:  "I appreciate you taking the time to share your concerns with me. I'm sorry to hear that you're not satisfied with your experience. Can you please provide more details about what went wrong so I can better understand the issue and see how I can improve it for you in the future?"


In [7]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
  +-----------------+    
  | ChatHuggingFace |    
  +-----------------+    
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     
